# Connect DB

In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine.reflection import Inspector



NEWS_DB_URL = f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{NEWS_DB_DATABASE}?charset=utf8mb4'

engine = create_engine(NEWS_DB_URL, echo=True)  # echo=True will turn on the logging of the actual SQL queries

In [2]:
# 테이블 목록 조회
inspector = Inspector.from_engine(engine)
table_names = inspector.get_table_names()
table_names

2024-03-20 15:18:31,078 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-03-20 15:18:31,078 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-20 15:18:31,082 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-03-20 15:18:31,082 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-20 15:18:31,084 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-03-20 15:18:31,084 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-20 15:18:31,089 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 15:18:31,089 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `portal_news_scraper`
2024-03-20 15:18:31,090 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-20 15:18:31,093 INFO sqlalchemy.engine.Engine ROLLBACK


/tmp/ipykernel_3560070/142063905.py:2: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


['new_company_info',
 'news_category_code',
 'portal_news',
 'portal_news_2021',
 'portal_news_2022',
 'portal_news_2023']

In [4]:
news_table_names = [
    'portal_news',
    'portal_news_2021',
    'portal_news_2022',
    'portal_news_2023'
    ]

In [8]:
# 각 테이블에서 content 컬럼의 데이터 조회
for table_name in news_table_names:
    print(f'테이블: {table_name}')
    with engine.connect() as con:
        rs = con.execute(text(f'SELECT content FROM {table_name} LIMIT 1'))
        for row in rs:
            if row[0] is None:
                print('content 컬럼이 비어 있습니다.')
            else:
                print(row[0][:100])  # content 컬럼의 내용 중 앞 100자만 출력
    print()  # 테이블 간 구분을 위해 빈 줄 출력

테이블: portal_news
2024-03-20 15:23:32,062 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 15:23:32,063 INFO sqlalchemy.engine.Engine SELECT content FROM portal_news LIMIT 1
2024-03-20 15:23:32,064 INFO sqlalchemy.engine.Engine [cached since 191s ago] {}
현대건설. 한국배구연맹
           

프로배구 여자부 현대건설이 압도적인 높이를 앞세워 1위를 수성했다.
현대건설은 31일 인천 삼산월드체육관에서 열린 2023-2024시
2024-03-20 15:23:32,067 INFO sqlalchemy.engine.Engine ROLLBACK

테이블: portal_news_2021
2024-03-20 15:23:32,069 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 15:23:32,069 INFO sqlalchemy.engine.Engine SELECT content FROM portal_news_2021 LIMIT 1
2024-03-20 15:23:32,069 INFO sqlalchemy.engine.Engine [cached since 191s ago] {}
content 컬럼이 비어 있습니다.
2024-03-20 15:23:32,071 INFO sqlalchemy.engine.Engine ROLLBACK

테이블: portal_news_2022
2024-03-20 15:23:32,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 15:23:32,073 INFO sqlalchemy.engine.Engine SELECT content FROM portal_news_2022 LIMIT 1
2024-03-20 15:23:32,07

# Cleansing Data

In [5]:
import re


def process_content(text):
    try:
        # e-mail 주소를 제거합니다.
        text = re.sub(r'\S+@\S+', '', text)
        # HTML 공백 문자인 &nbsp;를 실제 공백으로 대체합니다.
        text = re.sub(r'\u00A0', ' ', text)
        text = re.sub(r'&nbsp;', ' ', text)
        # 두 개 이상 연속된 공백을 하나의 공백으로 치환합니다.
        text = re.sub(r' {2,}', ' ', text)
        # 각 줄의 시작 부분에 있는 공백을 제거합니다.
        text = re.sub(r'(?m)^\s+', '', text)
        # 빈 줄을 제거합니다.
        text = re.sub(r'(?m)^\n', '', text)
        
        # 문자열의 앞뒤에 있는 모든 공백을 제거합니다.
        return text.strip()
    except Exception as e:
        print('content 전처리 중 에러 발생:', e)
        return text

In [12]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Text, DateTime
from sqlalchemy.dialects.mysql import BIGINT

Base = declarative_base()


class PortalNews(Base):
    """다음 뉴스 모델 클래스"""

    __tablename__ = 'portal_news'

    id = Column(BIGINT, primary_key=True, autoincrement=True)
    url = Column(String(255))
    title = Column(String(255))
    content = Column(Text)
    create_date = Column(DateTime)
    kind = Column(String(10))
    url_md5 = Column(String(35), unique=True)
    image_url = Column(Text)
    portal = Column(String(255))
    media = Column(String(255))
    norm_title = Column(String(255))

    # 테이블 인코딩 설정
    __table_args__ = {
        'mysql_charset': 'utf8mb4',
        'mysql_collate': 'utf8mb4_unicode_ci'
        }


class EsgNews(Base):
    """ESG 뉴스 모델 클래스"""

    __tablename__ = 'esg_news'

    id = Column(BIGINT, primary_key=True, autoincrement=True)
    url = Column(String(255))
    title = Column(String(255))
    content = Column(Text)
    create_date = Column(DateTime)
    kind = Column(String(10))
    url_md5 = Column(String(35), unique=True)
    image_url = Column(Text)
    portal = Column(String(255))
    media = Column(String(255))
    esg_analysis = Column(Text)
    norm_title = Column(String(255))

    # 테이블 인코딩 설정
    __table_args__ = {
        'mysql_charset': 'utf8mb4',
        'mysql_collate': 'utf8mb4_unicode_ci'
        }


class EtcNews(Base):
    """기타 뉴스 모델 클래스"""

    __tablename__ = 'etc_news'

    id = Column(BIGINT, primary_key=True, autoincrement=True)
    url = Column(String(255))
    title = Column(String(255))
    content = Column(Text)
    create_date = Column(DateTime)
    kind = Column(String(10))
    url_md5 = Column(String(35), unique=True)
    image_url = Column(Text)
    portal = Column(String(255))
    media = Column(String(255))
    norm_title = Column(String(255))

    # 테이블 인코딩 설정
    __table_args__ = {
        'mysql_charset': 'utf8mb4',
        'mysql_collate': 'utf8mb4_unicode_ci'
        }


class NaverNews(Base):
    """네이버 뉴스 모델 클래스"""

    __tablename__ = 'naver_news'

    id = Column(BIGINT, primary_key=True, autoincrement=True)
    url = Column(String(255))
    title = Column(String(255))
    content = Column(Text)
    create_date = Column(DateTime)
    kind = Column(String(10))
    url_md5 = Column(String(35), unique=True)
    image_url = Column(Text)
    portal = Column(String(255))
    media = Column(String(255))
    norm_title = Column(String(255))

    # 테이블 인코딩 설정
    __table_args__ = {
        'mysql_charset': 'utf8mb4',
        'mysql_collate': 'utf8mb4_unicode_ci'
        }

/tmp/ipykernel_3500378/3843516001.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [13]:
model_map = {
    'daum_news': DaumNews,
    'esg_news': EsgNews,
    'etc_news': EtcNews,
    'naver_news': NaverNews,
    }

In [14]:
from sqlalchemy.orm import sessionmaker
import pandas as pd
from tqdm.auto import tqdm 
tqdm.pandas()


Session = sessionmaker(bind=engine)
session = Session()

for table_name in news_table_names:
    # 데이터베이스에서 데이터를 가져옵니다.
    df = pd.read_sql(f'SELECT id, content FROM {table_name}', con=engine)

    # 'content' 컬럼을 process_content 함수로 처리합니다.
    tqdm.pandas(desc=f"Processing {table_name}")
    df['content'] = df['content'].progress_apply(process_content)

    # bulk_update_mappings을 사용하여 업데이트합니다.
    session.bulk_update_mappings(
        model_map[table_name],
        df.to_dict(orient='records')
    )
    session.commit()

    print(f'테이블 {table_name} 업데이트 완료')

session.close()


2024-03-20 15:10:57,009 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 15:10:57,009 INFO sqlalchemy.engine.Engine DESCRIBE `news_scraper`.`SELECT id, content FROM daum_news`
2024-03-20 15:10:57,009 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-20 15:10:57,013 INFO sqlalchemy.engine.Engine SELECT id, content FROM daum_news
2024-03-20 15:10:57,014 INFO sqlalchemy.engine.Engine [raw sql] {}


/home/ruo/.cache/pypoetry/virtualenvs/test-1GvpDNxc-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-03-20 15:10:57,339 INFO sqlalchemy.engine.Engine ROLLBACK


Processing daum_news: 100%|██████████| 7537/7537 [00:00<00:00, 19995.93it/s]

2024-03-20 15:10:57,730 INFO sqlalchemy.engine.Engine BEGIN (implicit)


...

2024-03-20 15:11:10,070 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 15:11:10,072 INFO sqlalchemy.engine.Engine UPDATE etc_news SET content=%(content)s WHERE etc_news.id = %(etc_news_id)s
2024-03-20 15:11:10,072 INFO sqlalchemy.engine.Engine [generated in 0.00071s] [{'content': '과학기술정보통신부장관 이종호 이창윤 제1차관은 우주항공청 사천 임시청사를 방문 입주 준비 상황을 점검했다\n이 차관은 이어 사천시장과 경남도청 경제부지사 등 관계자들을 만나 협력 방안을 논의했다우주항공청이 업무 공간으로 사용할 임시청사는 경남 사천시 사남면 사천제2일 ... (258 characters truncated) ... 했다\n관련기사\n우주청 임무본부장 연봉 대통령급20240314\n한국판 NASA 우주항공청 만든다이르면 5월 출범20240109\n우주항공청 특별법 국회 과방위 문턱 넘었다20240108\n항우연천문연 역대 원장 정쟁 때문에 우주항공청 설립 지연 안돼20231130', 'etc_news_id': 15545}, {'content': '편집자주 이사회는 회사와 주주의 이익을 위해 최종적으로 의사결정을 내리는 조직이다 경영전략은 물론 재무 인사 등 회사의 미래를 결정지을 법한 의안들을 다룬다 각사의 이사회가 한 해 동안 다룬 주요 의안들을 보면 그 회사의 미래 지향점이 어디인지 또 당장 어디에 경영  ... (1998 characters truncated) ... 자를 늘려간다 지난달 재무위원회 주재로 열린 자리에서 CARBONCO 유상증자 참여 승인의 건이 가결된 게 DL이앤씨의 현 기조를 방증한다 출자 규모는 150억원으로 알려졌다 카본코의 성장 궤도에 따라 플랜트사업본부는 보다 다양한 포트폴리오를 확보할 수 있을 전망이

2024-03-20 15:11:10,484 INFO sqlalchemy.engine.Engine COMMIT
테이블 etc_news 업데이트 완료
2024-03-20 15:11:10,496 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 15:11:10,497 INFO sqlalchemy.engine.Engine DESCRIBE `news_scraper`.`SELECT id, content FROM naver_news`
2024-03-20 15:11:10,497 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-20 15:11:10,500 INFO sqlalchemy.engine.Engine SELECT id, content FROM naver_news
2024-03-20 15:11:10,501 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-20 15:11:11,260 INFO sqlalchemy.engine.Engine ROLLBACK


Processing naver_news: 100%|██████████| 19369/19369 [00:00<00:00, 20357.95it/s]

2024-03-20 15:11:12,229 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2024-03-20 15:11:12,334 INFO sqlalchemy.engine.Engine UPDATE naver_news SET content=%(content)s WHERE naver_news.id = %(naver_news_id)s
2024-03-20 15:11:12,334 INFO sqlalchemy.engine.Engine [generated in 0.05907s] [{'content': '선거대책위에서 직책맡으며 지지세 확장 도움\n410 총선 공천 티켓을 놓고 경쟁을 벌였던 같은 당 예비후보들이 선당후사의 자세로 다시 힘을 모으고 있다\n춘천지역을 중심으로 움직였던 국민의힘 예비후보들은 김혜란 춘천갑 예비후보를 지지하며 캠프에 합류했다 춘천갑에서 공 ... (498 characters truncated) ... 는 공천 과정에서 컷오프됐던 이종석 예비후보가 민주당을 탈당했으며 무소속 출마를 고민중인 것으로 알려졌다\n공천에서 배제됐던 도내 한 국회의원 예비후보는 당의 승리를 위해 경쟁했던 후보의 당선에 보탬이 되어야 하는 것은 맞지만 아직 마음의 결정을 내리지 못했다고 했다', 'naver_news_id': 13728811}, {'content': '더불어민주당 전성 춘천철원화천양구을 국회의원 예비후보가 공보의 상급병원 차출로 지역의료 공백이 심각해지고 있다며 정부를 향해 비판의 목소리를 냈다\n전 예비후보는 18일 자료를 내고 지난주 화천군에서만 공보의 3명을 파견했고 춘천 사북면보건지소는 진료 횟수를 주 3회 ... (232 characters truncated) ... 꼬집었다\n그는 지역구 주민들의 심각한 의료위기를 외면하고 있는 한기호 국회의원의 책임도 묻지 않을 수 없다며 국회의원이 되면 응급의료기관 접근권을 강화하고 접경지역을 의료위기지역으로 지정해 방문진료 약배달 등 저위험 의사 업무를 허용하는 방안을 추진하겠다고 강조했다', 'naver_news_id': 13728812}, {'content': '당 지도부 후보 등록일까지 바